In [ ]:
import sys
sys.path.append("..")

import format_data_helpers as fdh

In [ ]:
slowslowfolder = "20230222_194413"
fastslowfolder = "20230222_171344"
slowfastfolder = "20230222_182149"

# the first list of "bad" indices are the ones known when recording to have dropped the object or otherwise 
# messed up during recording
# second list is manual validation looking at cleaned data
bad_slowslow_trajectories = [5,32,35,43,81,99,114,117,121] + [36,41,46,108,105] 
bad_fastslow_trajectories = [4,18,57,68,94,96,99,108,128] + [45]
bad_slowfast_trajectories = [4,9,12,19,31,86,98,107,130] 

In [ ]:
slowslowrawimg, slowslowrawdat, slowslowinds = fdh.load_trajectories_as_list(f"../forkdata/{slowslowfolder}",bad_slowslow_trajectories, max_traj_number=140)
fastslowrawimg, fastslowrawdat, fastslowinds = fdh.load_trajectories_as_list(f"../forkdata/{fastslowfolder}",bad_fastslow_trajectories, max_traj_number=140)
slowfastrawimg, slowfastrawdat, slowfastinds = fdh.load_trajectories_as_list(f"../forkdata/{slowfastfolder}",bad_slowfast_trajectories, max_traj_number=140)


In [ ]:
import cv2

from get_string_pixel_parameters import process_image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def get_bad_identifications(folder, allinds):
    xmin, xmax = (45,210)
    ymin, ymax = (75,245)

    bad_inds = []
    for i in allinds:
        filename = f"../forkdata/{folder}/overhead_image_{i}.png"
        approx_diam = 168 # mm
        plate_radius_pixels = 77

        src = cv2.imread(filename)

        (plate_center, plate_radius),ellipse = process_image(filename)

        if ellipse[1][0] < 60: # if longest is less than 60, then it's probably got the string nicely
            continue
        print(f"For image {i}")
        print("ellipse area proportional to:", ellipse[1][0]*ellipse[1][1])
        bad_inds.append(i)
        
        #cv2.circle(src, (plate_center[0],plate_center[1]), plate_radius_pixels, (1,1,1), 1)
        #plt.imshow(src[xmin:xmax,ymin:ymax,::-1].transpose((1,0,2))[::-1,:,:])
#
        #cv2.ellipse(src, ellipse, (1,1,255), -1)
        #plt.imshow(src[xmin:xmax,ymin:ymax,::-1][::-1,::-1,:])
        #plt.axis("off")
        #physx_frame_x_loc = -(ellipse[0][0]-plate_center[0]) / plate_radius_pixels * approx_diam/2
        #physx_frame_y_loc = (ellipse[0][1]-plate_center[1]) / plate_radius_pixels * approx_diam/2
        #physx_frame_theta = (np.pi - ellipse[2]/180*np.pi)
        #print(physx_frame_x_loc, physx_frame_y_loc,physx_frame_theta)
        #plt.show()
        #plt.figure(figsize=(4,4))
        #ts = np.linspace(0,np.pi*2,200)
        #xs = np.cos(ts) * approx_diam/2
        #ys = np.sin(ts) * approx_diam/2
        #plt.plot(xs,ys)
        #stringx = [physx_frame_x_loc,physx_frame_x_loc+np.cos(physx_frame_theta)*10,physx_frame_x_loc-np.cos(physx_frame_theta)*10]
        #stringy = [physx_frame_y_loc,physx_frame_y_loc+np.sin(physx_frame_theta)*10,physx_frame_y_loc-np.sin(physx_frame_theta)*10]
        #plt.scatter(stringx,stringy,s=10)
        #plt.axis("equal")
        #plt.axis("off")
        #plt.show()
    return bad_inds

In [ ]:

visually_bad_slowslowinds = get_bad_identifications(slowslowfolder,slowslowinds)
visually_bad_slowfastinds = get_bad_identifications(slowfastfolder,slowfastinds)
visually_bad_fastslowinds = get_bad_identifications(fastslowfolder,fastslowinds)

In [ ]:
## These bad indices were determined based on bad trajectories (eg: recorded not a full pickup or spaghetti fell)

ok_slowslow_trajectories = [i not in bad_slowslow_trajectories and i not in visually_bad_slowslowinds for i in slowslowinds]
slowslow_kept_indices = np.array([f"ss{i}" for i in slowslowinds if i not in bad_slowslow_trajectories and i not in visually_bad_slowslowinds ])
slowslowdat = slowslowrawdat[ok_slowslow_trajectories]
slowslowimg = slowslowrawimg[ok_slowslow_trajectories]


ok_fastslow_trajectories = [i not in bad_fastslow_trajectories and i not in visually_bad_fastslowinds for i in fastslowinds]
fastslow_kept_indices = np.array([f"fs{i}" for i in fastslowinds if i not in bad_fastslow_trajectories and i not in visually_bad_fastslowinds ])
fastslowdat = fastslowrawdat[ok_fastslow_trajectories]
fastslowimg = fastslowrawimg[ok_fastslow_trajectories]

ok_slowfast_trajectories = [i not in bad_slowfast_trajectories and i not in visually_bad_slowfastinds for i in slowfastinds]
slowfast_kept_indices = np.array([f"sf{i}" for i in slowfastinds if i not in bad_slowfast_trajectories and i not in visually_bad_slowfastinds])
slowfastdat = slowfastrawdat[ok_slowfast_trajectories]
slowfastimg = slowfastrawimg[ok_slowfast_trajectories]
print(slowslowdat.shape)
print(fastslowdat.shape)
print(slowfastdat.shape)

print(len(slowslow_kept_indices))
print(len(fastslow_kept_indices))
print(len(slowfast_kept_indices))


In [ ]:
# only keep first 115 of each kind of data
number_to_keep = 115
slowslowdat = slowslowdat[:number_to_keep]
slowslowimg = slowslowimg[:number_to_keep]
slowslow_kept_indices = slowslow_kept_indices[:number_to_keep]
fastslowdat = fastslowdat[:number_to_keep]
fastslowimg = fastslowimg[:number_to_keep]
fastslow_kept_indices = fastslow_kept_indices[:number_to_keep]
slowfastdat = slowfastdat[:number_to_keep]
slowfastimg = slowfastimg[:number_to_keep]
slowfast_kept_indices = slowfast_kept_indices[:number_to_keep]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
gen = np.random.default_rng(seed=0)

In [ ]:
all_pose_data = np.concatenate((slowfastdat, fastslowdat, slowslowdat),axis=0)
train_pose_data_inds = gen.choice(len(all_pose_data), size=240, replace=False) #LOL forgot replace=False last time
test_pose_data_inds = np.array([i for i in range(len(all_pose_data)) if i not in train_pose_data_inds])
train_pose_data = all_pose_data[train_pose_data_inds]
test_pose_data = all_pose_data[test_pose_data_inds]

all_image_info = np.concatenate((slowfastimg,fastslowimg,slowslowimg),axis=0)
train_image_info = all_image_info[train_pose_data_inds]
test_image_info = all_image_info[test_pose_data_inds]

all_names_info = np.concatenate((slowfast_kept_indices,fastslow_kept_indices,slowslow_kept_indices),axis=0)
train_names_info = all_names_info[train_pose_data_inds]
test_names_info = all_names_info[test_pose_data_inds]


In [ ]:
# the mean of all x,y,z,rw,rx,ry,rz pointwise (over all trajectories)
train_pose_data_mean = np.mean(train_pose_data, axis=(0,1)).reshape(1,7)
train_pose_data_centered = train_pose_data - train_pose_data_mean
position_std = np.sqrt(np.var(train_pose_data_centered[:,:,:3]))
position_scale = 1./position_std
rotation_scale = position_scale * 0.08
train_pose_data_scaling = np.array((position_scale, position_scale, position_scale, rotation_scale,rotation_scale,rotation_scale, rotation_scale)).reshape(1,7)
train_pose_data_scaled = train_pose_data_centered * train_pose_data_scaling

test_pose_data_centered = test_pose_data - train_pose_data_mean
test_pose_data_scaled = test_pose_data_centered * train_pose_data_scaling

datapath = "../forkdata/forkTrajectoryData.npz"


# weirdly the train_image_info was giving precision errors comparing labgpu run from local run
# in order to get sha256sums to tie out regardless of where this is run, don't save image info
np.savez(datapath,
        #train_image_info = train_image_info,
        train = train_pose_data_scaled,
        #test_image_info=test_image_info,
        test = test_pose_data_scaled,
        pose_scaling = train_pose_data_scaling,
        pose_mean = train_pose_data_mean,
        train_names_info = train_names_info,
        test_names_info = test_names_info)


In [ ]:
train_pose_data_centered[:,:,:3].shape

In [ ]:
np.sqrt(np.var(train_pose_data_scaled[:,:3]))

In [ ]:
np.sqrt(np.mean(np.var(train_pose_data_scaled[:,:3],axis=(0,1))))

In [ ]:
np.mean(np.sum(np.square(train_pose_data_scaled[:,:,:3]),axis=2))

In [ ]:
train_pose_data_scaled.shape

In [ ]:
test_pose_data_scaled.shape